In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import sqlalchemy as sa

import database as db
from images import copy_images, get_list_of_images_invalid, save_metadata
from models import County, DataTrustedIdentifier, Image
from unaccent import unaccent

In [3]:
engine, session=db.connect()
engine.echo=False

2023-03-30 15:21:02,134 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-30 15:21:02,135 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-30 15:21:02,136 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-30 15:21:02,136 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-30 15:21:02,137 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-30 15:21:02,138 INFO sqlalchemy.engine.Engine [raw sql] {}


In [4]:
query = session.query(County).distinct().all()

list_uf = [unaccent(sa.func.lower(q.uf)) for q in query]
list_state = [unaccent(sa.func.lower(q.state)) for q in query]
list_county = [unaccent(sa.func.lower(q.county)) for q in query]

uf_unaccented_lower = unaccent(sa.func.lower(DataTrustedIdentifier.state_province)).in_(list_uf)
state_unaccented_lower = unaccent(sa.func.lower(DataTrustedIdentifier.state_province)).in_(list_state)
county_unaccented_lower = unaccent(sa.func.lower(DataTrustedIdentifier.county)).in_(list_county)

In [5]:
list_minimum_image=[5, 10, 20]
list_color=['rgb', 'grayscale']
list_image_size=['256', '400', '512']
path_out = '/home/xandao/Imagens/br_dataset'
list_images_invalid=get_list_of_images_invalid()
list_images_invalid=list_images_invalid['barcode']
list_images_invalid

['INPA0248526',
 'INPA0248523',
 'INPA0248528',
 'NY01421926_01',
 'NY01421575_01',
 'HUFSJ001689_v00',
 'HUFSJ001133_v00',
 'HUFSJ002198_v00',
 'HUFSJ003255_v00',
 'HVASF000487_v01',
 'HUFSJ001689_v01',
 'INPA0019084_nd',
 'INPA0022379_nd',
 'INPA0032742_nd',
 'INPA0023115',
 'NL-U1484137',
 'NY01397568_01',
 'INPA0012286',
 'INPA0146998']

In [6]:
for color in list_color:
    for image_size in list_image_size:
        for minimum_image in list_minimum_image:
            list_level = []
            list_path_images_final = []
            list_seq_final = []
            list_count_samples = []

            level=DataTrustedIdentifier.specific_epithet_trusted
            out=os.path.join(path_out, color.upper(), level.name, image_size, str(minimum_image))

            condition = sa.and_(DataTrustedIdentifier.country_trusted == 'Brasil',
                                level.is_not(None),
                                sa.or_(uf_unaccented_lower, state_unaccented_lower))

            columns = [level,
                       sa.func.array_agg(DataTrustedIdentifier.seq).label('list_seq')]
            query = session.query(*columns) \
                .filter(condition) \
                .distinct() \
                .group_by(level) \
                .order_by(level) \
                .having(sa.func.count(level) >= minimum_image) \
                .all()

            print('color: %s image_size: %s minimum_image: %d' % (color.upper(), image_size, minimum_image))
            for q in query:
                qzao = session.query(Image.seq_id, sa.func.array_agg(Image.path_segmented).label('list_path_segmented')) \
                    .filter(sa.and_(Image.seq_id.in_(q.list_seq),
                                    Image.color_mode.__eq__(color.upper()),
                                    Image.height.__eq__(image_size),
                                    Image.width.__eq__(image_size),
                                    sa.not_(Image.filename.in_(list_images_invalid)),
                                    sa.not_(Image.filename.ilike('%_v0%')),
                                    sa.not_(Image.filename.like('%_e%')),
                                    sa.not_(Image.filename.like('%_nd%')),
                                    ))\
                    .group_by(Image.seq_id)\
                    .all()

                list_one_image_per_seq = []
                list_seq_one_image = []
                for qzinho in qzao:
                    list_seq_one_image.append(qzinho.seq_id)
                    list_one_image_per_seq.append(sorted(qzinho.list_path_segmented)[0])

                if len(list_one_image_per_seq) >= minimum_image:
                    list_count_samples.append(len(list_one_image_per_seq))
                    list_level.append(q.specific_epithet_trusted)
                    list_path_images_final.append(list_one_image_per_seq)
                    list_seq_final.append(list_seq_one_image)

            list_f = copy_images(list_level, list_path_images_final, out)
            save_metadata(color, image_size, level, list_count_samples, list_f, list_level, list_path_images_final, list_seq_final, minimum_image, out, session)

/tmp/ipykernel_64996/2601300049.py:18: SAWarning: Class unaccent will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/20/cprf)
  query = session.query(*columns) \


color: RGB image_size: 256 minimum_image: 5
total of levels: 235 total of images: 10515


,levels,count,dst,paths,seq
0,abutiloides,8,/home/xandao/Imagens/br_dataset/RGB/specific_e...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[9416, 10009, 10651, 12428, 25112, 37910, 3847..."
1,aduncum,377,/home/xandao/Imagens/br_dataset/RGB/specific_e...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[7220, 7221, 7222, 7234, 7742, 7757, 8224, 868..."
2,aequale,30,/home/xandao/Imagens/br_dataset/RGB/specific_e...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[8727, 9316, 10163, 11517, 12577, 20200, 21125..."
3,alata,81,/home/xandao/Imagens/br_dataset/RGB/specific_e...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[6857, 6861, 6862, 6863, 6864, 7565, 7859, 986..."


,seq,genus,specific_epithet,genus_trusted,specific_epithet_trusted,urls
0,9735,Piper,arboreum,Piper,arboreum,[https://storage.googleapis.com/cria-zoomify/h...
1,19770,Ottonia,anisum,Piper,anisum,[https://storage.googleapis.com/cria-zoomify/h...
2,40891,Ottonia,taperana,Piper,anisum,[https://storage.googleapis.com/cria-zoomify/h...
3,7333,Piper,corcovadense,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...


,0
dst,/home/xandao/Imagens/br_dataset/RGB/specific_e...
color,rgb
image_size,256
minimum_image,5
level_name,specific_epithet_trusted
levels,235
total,10515
average,44.74


...

,0
dst,/home/xandao/Imagens/br_dataset/GRAYSCALE/spec...
color,grayscale
image_size,512
minimum_image,10
level_name,specific_epithet_trusted
levels,160
total,9991
average,62.44


color: GRAYSCALE image_size: 512 minimum_image: 20
total of levels: 106 total of images: 9248


,levels,count,dst,paths,seq
0,aduncum,377,/home/xandao/Imagens/br_dataset/GRAYSCALE/spec...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[7220, 7221, 7222, 7234, 7742, 7757, 8224, 868..."
1,aequale,30,/home/xandao/Imagens/br_dataset/GRAYSCALE/spec...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[8727, 9316, 10163, 11517, 12577, 20200, 21125..."
2,alata,81,/home/xandao/Imagens/br_dataset/GRAYSCALE/spec...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[6857, 6861, 6862, 6863, 6864, 7565, 7859, 986..."
3,alatabaccum,28,/home/xandao/Imagens/br_dataset/GRAYSCALE/spec...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[26207, 30555, 30557, 30558, 30560, 30561, 305..."


,seq,genus,specific_epithet,genus_trusted,specific_epithet_trusted,urls
0,9735,Piper,arboreum,Piper,arboreum,[https://storage.googleapis.com/cria-zoomify/h...
1,19770,Ottonia,anisum,Piper,anisum,[https://storage.googleapis.com/cria-zoomify/h...
2,40891,Ottonia,taperana,Piper,anisum,[https://storage.googleapis.com/cria-zoomify/h...
3,7333,Piper,corcovadense,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...


,0
dst,/home/xandao/Imagens/br_dataset/GRAYSCALE/spec...
color,grayscale
image_size,512
minimum_image,20
level_name,specific_epithet_trusted
levels,106
total,9248
average,87.25


In [7]:
session.close()
engine.dispose()